<a href="https://colab.research.google.com/github/makeshiftworks/russian_car_palte/blob/main/russian20250517.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
import json
f = open("/content/drive/MyDrive/kaggle/russian_car_plate/kaggle.json")
json_data = json.load(f)
os.environ['KAGGLE_USERNAME'] = json_data['username']
os.environ['KAGGLE_KEY'] = json_data['key']

!pip install kaggle
!kaggle competitions download -c russian-car-plates-prices-prediction
! unzip russian-car-plates-prices-prediction.zip
! ls -la ./

Mounted at /content/drive
  0% 0.00/742k [00:00<?, ?B/s]
100% 742k/742k [00:00<00:00, 1.20GB/s]
Archive:  russian-car-plates-prices-prediction.zip
  inflating: sample_submission.csv   
  inflating: supplemental_english.py  
  inflating: supplemental_russian.py  
  inflating: test.csv                
  inflating: train.csv               
total 3388
drwxr-xr-x 1 root root    4096 May 17 10:55 .
drwxr-xr-x 1 root root    4096 May 17 10:53 ..
drwxr-xr-x 4 root root    4096 May 14 13:38 .config
drwx------ 6 root root    4096 May 17 10:55 drive
-rw-r--r-- 1 root root  760133 Feb 27 17:56 russian-car-plates-prices-prediction.zip
drwxr-xr-x 1 root root    4096 May 14 13:38 sample_data
-rw-r--r-- 1 root root  107738 Feb 27 17:56 sample_submission.csv
-rw-r--r-- 1 root root   24422 Feb 27 17:56 supplemental_english.py
-rw-r--r-- 1 root root   31685 Feb 27 17:56 supplemental_russian.py
-rw-r--r-- 1 root root  282445 Feb 27 17:56 test.csv
-rw-r--r-- 1 root root 2233293 Feb 27 17:56 train.csv


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

def convert_to_year_week(date_obj):
    """
    object型の日付をyyyy-week形式に変換する関数。

    Parameters:
    date_obj: str or object, 日付（例: '2024-07-12 21:31:37'）

    Returns:
    str: yyyy-week形式の文字列（例: '2024-28'）
    """
    # object型を文字列として処理し、datetimeに変換
    if isinstance(date_obj, str):
        dt = datetime.strptime(date_obj, '%Y-%m-%d %H:%M:%S')
    else:
        dt = pd.to_datetime(date_obj)

    # 年、ISO週番号を取得
    year = dt.year
    week = dt.isocalendar().week

    # yyyy-week形式で返す（週番号は2桁ゼロ埋め）
    return f"{year}-{week:02d}"


def split_plate(df):
    #dateの型をobjectからdatetimeに変換
    df['year_week'] = df['date'].apply(convert_to_year_week)


    # plate要素を分割-一番最初のアルファベットを'1st',2番目の数字３桁を'2nd',3番目のアルファベット2文字を'3rd',に分割
    # plate要素を分割
    df['1st'] = df['plate'].str.extract(r'([A-Za-z])')[0]
    df['2nd'] = df['plate'].str.extract(r'(\d{3})')[0]
    df['3rd'] = df['plate'].str.extract(r'([A-Za-z]{2})')[0]
    # plate要素の最後から３文字の数字を4thに分割する。
    df['4th'] = df['plate'].str.extract(r'(\d{3})$')[0]

    #4thがNanの場合、plate要素の最後から２文字の数値を4thに入れる
    df['4th'] = df['4th'].fillna(train['plate'].str.extract(r'(\d{2})$')[0])


    return df


train = split_plate(train)
test = split_plate(test)

test

,id,plate,date,price,year_week,1st,2nd,3rd,4th
0,51636,P700TT790,2025-01-27 00:00:00,NaN,2025-05,P,700,TT,790
1,51637,M081TX797,2025-02-10 00:00:00,NaN,2025-07,M,081,TX,797
2,51638,T333HX777,2025-02-11 00:00:00,NaN,2025-07,T,333,HX,777
3,51639,H744BH977,2025-02-03 00:00:00,NaN,2025-06,H,744,BH,977
4,51640,X066EM777,2025-02-12 00:00:00,NaN,2025-07,X,066,EM,777
...,...,...,...,...,...,...,...,...,...
7690,59326,X799CC799,2023-03-13 00:00:00,NaN,2023-11,X,799,CC,799
7691,59327,K077YT777,2022-10-19 00:00:00,NaN,2022-42,K,077,YT,777
7692,59328,O200PA777,2023-10-02 19:19:30,NaN,2023-40,O,200,PA,777
7693,59329,O073OO97,2025-01-12 00:00:00,NaN,2025-02,O,073,OO,97


In [ ]:
# beautiful_number

# beautiful_number: 1
# prompt: train['2nd"]とtrain['4th]が同じ文字のデータを抽出。グループ化して2ndと4thとpriceの中央値を一覧表示。（priceで降順)

# '2nd'と'4th'が同じ値のデータを抽出
same_2nd_4th = train[train['2nd'] == train['4th']]

# グループ化して集計
grouped = same_2nd_4th.groupby(['2nd', '4th'])['price'].median().reset_index()

# priceで降順にソート
bnumber1 = grouped.sort_values('price', ascending=False)



# beautiful_number: 2
# prompt: train['4th']が2文字の場合、train['4th']と同じ文字がtrain['2nd']に含まれているデータを抽出。グループ化して2ndと4thとpriceの中央値を一覧表示。（priceで降順)

# '4th'が2文字のデータに絞り込み
train_filtered = train[train['4th'].str.len() == 2]

# '4th'の値が'2nd'に含まれるデータを抽出
filtered_data = train_filtered[train_filtered.apply(lambda row: row['4th'] in row['2nd'], axis=1)]

# グループ化して集計
grouped = filtered_data.groupby(['2nd', '4th'])['price'].median().reset_index()

# priceで降順にソート
bnumber2 = grouped.sort_values('price', ascending=False)



# prompt: # prompt: bnumber1の下にbnumber2を結合しbeautiful_numberというdataframeにする。

beautiful_number = pd.concat([bnumber1, bnumber2]).reset_index(drop=True)

# Rename the 'price' column to 'bn_price'
beautiful_number = beautiful_number.rename(columns={'price': 'bn_price'})
beautiful_number



,2nd,4th,bn_price
0,777,777,4400000.0
1,155,155,1600000.0
2,177,177,450000.0
3,161,161,425000.0
4,163,163,400000.0
...,...,...,...
381,334,33,32500.0
382,795,95,32500.0
383,955,95,30000.0
384,964,64,30000.0


In [ ]:
# highDemand_letter

# prompt: train['1st']とtrain['3rd']の組合せで集計してpriceの中央値を降順で表示

# '1st'と'3rd'の組み合わせでグループ化し、'price'の中央値を計算
grouped = train.groupby(['1st', '3rd'])['price'].median().reset_index()

# 'price'で降順にソート
highDemand_letter = grouped.sort_values('price', ascending=False)

# Rename the 'price' column to 'hdl_price'
highDemand_letter = highDemand_letter.rename(columns={'price': 'hdl_price'})

# 結果を表示
highDemand_letter


,1st,3rd,hdl_price
800,K,MP,1200000.0
0,A,AA,1000000.0
622,H,EX,975000.0
942,M,MM,970000.0
80,A,MP,830000.0
...,...,...,...
1488,X,HA,50000.0
1290,P,YM,50000.0
192,B,HA,50000.0
221,B,MK,40000.0


In [ ]:
# prompt: train['2nd']をグループ化してpriceの中央値を降順で表示

# '2nd'でグループ化し、'price'の中央値を計算
grouped = train.groupby('2nd')['price'].median().reset_index()

# 'price'で降順にソート
highDemand_number = grouped.sort_values('price', ascending=False)

# Rename the 'price' column to 'hdn_price'
highDemand_number = highDemand_number.rename(columns={'price': 'hdn_price'})

highDemand_number


,2nd,hdn_price
757,758,800000.0
858,859,754000.0
948,949,730000.0
776,777,700000.0
578,579,650000.0
...,...,...
981,982,35000.0
234,235,30000.0
871,872,30000.0
831,832,30000.0


In [ ]:
# beautiful_numberと結合→bn_price

# Merge 'train' with 'beautiful_number' based on '2nd' and '4th' columns
if 'bn_price' in train.columns:
    train = train.drop(columns=['bn_price'])

train = pd.merge(train, beautiful_number, on=['2nd', '4th'], how='left')


# Display the updated 'train' DataFrame
train


,id,plate,date,price,year_week,1st,2nd,3rd,4th,bn_price
0,1,X059CP797,2024-12-26 00:00:00,65000,2024-52,X,059,CP,797,NaN
1,2,Y800MH790,2024-07-12 21:31:37,100000,2024-28,Y,800,MH,790,NaN
2,3,A212TX77,2024-04-18 00:00:00,290000,2024-16,A,212,TX,77,NaN
3,4,P001AY199,2025-01-03 00:27:15,680000,2025-01,P,001,AY,199,NaN
4,5,P001AY199,2025-01-10 09:32:41,750000,2025-02,P,001,AY,199,NaN
...,...,...,...,...,...,...,...,...,...,...
51630,51631,X023PP797,2025-01-29 00:00:00,70000,2025-05,X,023,PP,797,NaN
51631,51632,M004KA161,2025-01-31 00:00:00,1600000,2025-05,M,004,KA,161,NaN
51632,51633,E888EB199,2025-02-06 00:00:00,850000,2025-06,E,888,EB,199,NaN
51633,51634,X023XK77,2024-04-24 11:30:07,150000,2024-17,X,023,XK,77,NaN


In [ ]:
# highDemand_letterと結合→hdl_price

# Create a combined column for merging
train['1st_3rd'] = train['1st'] + train['3rd']
highDemand_letter['1st_3rd'] = highDemand_letter['1st'] + highDemand_letter['3rd']

# Merge 'train' with 'highDemand_letter' based on the combined column
if 'hdl_price' in train.columns:
    train = train.drop(columns=['hdl_price'])

train = pd.merge(train, highDemand_letter[['1st_3rd', 'hdl_price']], on='1st_3rd', how='left')

# Drop the temporary combined column
train = train.drop(columns=['1st_3rd'])
highDemand_letter = highDemand_letter.drop(columns=['1st_3rd'])

# Display the updated 'train' DataFrame
train


,id,plate,date,price,year_week,1st,2nd,3rd,4th,bn_price,hdl_price
0,1,X059CP797,2024-12-26 00:00:00,65000,2024-52,X,059,CP,797,NaN,60000.0
1,2,Y800MH790,2024-07-12 21:31:37,100000,2024-28,Y,800,MH,790,NaN,180000.0
2,3,A212TX77,2024-04-18 00:00:00,290000,2024-16,A,212,TX,77,NaN,90000.0
3,4,P001AY199,2025-01-03 00:27:15,680000,2025-01,P,001,AY,199,NaN,100000.0
4,5,P001AY199,2025-01-10 09:32:41,750000,2025-02,P,001,AY,199,NaN,100000.0
...,...,...,...,...,...,...,...,...,...,...,...
51630,51631,X023PP797,2025-01-29 00:00:00,70000,2025-05,X,023,PP,797,NaN,170000.0
51631,51632,M004KA161,2025-01-31 00:00:00,1600000,2025-05,M,004,KA,161,NaN,150000.0
51632,51633,E888EB199,2025-02-06 00:00:00,850000,2025-06,E,888,EB,199,NaN,110000.0
51633,51634,X023XK77,2024-04-24 11:30:07,150000,2024-17,X,023,XK,77,NaN,175000.0


In [ ]:
# highDemand_numberと結合→hdn_price

# Merge 'train' with 'highDemand_number' based on the '2nd' column
if 'hdn_price' in train.columns:
    train = train.drop(columns=['hdn_price'])

train = pd.merge(train, highDemand_number, on='2nd', how='left')

# Display the updated 'train' DataFrame
train

,id,plate,date,price,year_week,1st,2nd,3rd,4th,bn_price,hdl_price,hdn_price
0,1,X059CP797,2024-12-26 00:00:00,65000,2024-52,X,059,CP,797,NaN,60000.0,60000.0
1,2,Y800MH790,2024-07-12 21:31:37,100000,2024-28,Y,800,MH,790,NaN,180000.0,140000.0
2,3,A212TX77,2024-04-18 00:00:00,290000,2024-16,A,212,TX,77,NaN,90000.0,80000.0
3,4,P001AY199,2025-01-03 00:27:15,680000,2025-01,P,001,AY,199,NaN,100000.0,635000.0
4,5,P001AY199,2025-01-10 09:32:41,750000,2025-02,P,001,AY,199,NaN,100000.0,635000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
51630,51631,X023PP797,2025-01-29 00:00:00,70000,2025-05,X,023,PP,797,NaN,170000.0,120000.0
51631,51632,M004KA161,2025-01-31 00:00:00,1600000,2025-05,M,004,KA,161,NaN,150000.0,200000.0
51632,51633,E888EB199,2025-02-06 00:00:00,850000,2025-06,E,888,EB,199,NaN,110000.0,480000.0
51633,51634,X023XK77,2024-04-24 11:30:07,150000,2024-17,X,023,XK,77,NaN,175000.0,120000.0


In [ ]:
# prompt: year_weekごとにグループ化してpriceの中央値を計算

# trainデータをyear_weekごとにグループ化し、priceの中央値を計算
median_price_by_year_week = train.groupby('year_week')['price'].median().reset_index()
# Rename the 'price' column to 'hdl_price'
median_price_by_year_week = median_price_by_year_week.rename(columns={'price': 'week_price'})

# 結果を表示
median_price_by_year_week

,year_week,week_price
0,2021-07,132500.0
1,2021-08,125000.0
2,2021-09,350000.0
3,2021-10,85000.0
4,2021-11,100000.0
...,...,...
204,2025-03,90000.0
205,2025-04,150000.0
206,2025-05,110000.0
207,2025-06,130000.0


In [ ]:
# prompt: trainのyear_weekとmedian_price_by_year_weekを結合

# trainデータにweek_priceを結合
if 'week_price' in train.columns:
    train = train.drop(columns=['week_price'])

train = pd.merge(train, median_price_by_year_week, on='year_week', how='left')

# 更新されたtrainデータフレームを表示
train

,id,plate,date,price,year_week,1st,2nd,3rd,4th,bn_price,hdl_price,hdn_price,week_price
0,1,X059CP797,2024-12-26 00:00:00,65000,2024-52,X,059,CP,797,NaN,60000.0,60000.0,180000.0
1,2,Y800MH790,2024-07-12 21:31:37,100000,2024-28,Y,800,MH,790,NaN,180000.0,140000.0,220000.0
2,3,A212TX77,2024-04-18 00:00:00,290000,2024-16,A,212,TX,77,NaN,90000.0,80000.0,215000.0
3,4,P001AY199,2025-01-03 00:27:15,680000,2025-01,P,001,AY,199,NaN,100000.0,635000.0,50000.0
4,5,P001AY199,2025-01-10 09:32:41,750000,2025-02,P,001,AY,199,NaN,100000.0,635000.0,90000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51630,51631,X023PP797,2025-01-29 00:00:00,70000,2025-05,X,023,PP,797,NaN,170000.0,120000.0,110000.0
51631,51632,M004KA161,2025-01-31 00:00:00,1600000,2025-05,M,004,KA,161,NaN,150000.0,200000.0,110000.0
51632,51633,E888EB199,2025-02-06 00:00:00,850000,2025-06,E,888,EB,199,NaN,110000.0,480000.0,130000.0
51633,51634,X023XK77,2024-04-24 11:30:07,150000,2024-17,X,023,XK,77,NaN,175000.0,120000.0,195500.0


In [ ]:
# prompt: bn_priceの値がNanのデータには150000を入れて欠損値補完する

train['bn_price'] = train['bn_price'].fillna(0)
train

,id,plate,date,price,year_week,1st,2nd,3rd,4th,bn_price,hdl_price,hdn_price,week_price
0,1,X059CP797,2024-12-26 00:00:00,65000,2024-52,X,059,CP,797,0.0,60000.0,60000.0,180000.0
1,2,Y800MH790,2024-07-12 21:31:37,100000,2024-28,Y,800,MH,790,0.0,180000.0,140000.0,220000.0
2,3,A212TX77,2024-04-18 00:00:00,290000,2024-16,A,212,TX,77,0.0,90000.0,80000.0,215000.0
3,4,P001AY199,2025-01-03 00:27:15,680000,2025-01,P,001,AY,199,0.0,100000.0,635000.0,50000.0
4,5,P001AY199,2025-01-10 09:32:41,750000,2025-02,P,001,AY,199,0.0,100000.0,635000.0,90000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51630,51631,X023PP797,2025-01-29 00:00:00,70000,2025-05,X,023,PP,797,0.0,170000.0,120000.0,110000.0
51631,51632,M004KA161,2025-01-31 00:00:00,1600000,2025-05,M,004,KA,161,0.0,150000.0,200000.0,110000.0
51632,51633,E888EB199,2025-02-06 00:00:00,850000,2025-06,E,888,EB,199,0.0,110000.0,480000.0,130000.0
51633,51634,X023XK77,2024-04-24 11:30:07,150000,2024-17,X,023,XK,77,0.0,175000.0,120000.0,195500.0


In [ ]:
#カラム作成
X = train[['bn_price','hdl_price','hdn_price','week_price']]
y = train[['price']]

#ダミー変数に変換はなし


#まずは標準化
from sklearn.preprocessing import StandardScaler

def custom_scale(X, target_scales):
    """
    各カラムをStandardScalerで標準化し、指定されたスケールに調整。

    Parameters:
    X: pandas DataFrame or numpy array, 入力データ
    target_scales: dict, カラム名（またはインデックス）と目標スケールのマッピング
                   例: {'column1': 10, 'column2': 5}

    Returns:
    pandas DataFrame or numpy array, スケール調整済みのデータ
    """
    # 入力がDataFrameかnumpy配列か判定
    is_dataframe = isinstance(X, pd.DataFrame)
    if is_dataframe:
        X_array = X.values
        columns = X.columns
    else:
        X_array = X
        columns = range(X_array.shape[1])

    # StandardScalerで標準化（平均0、分散1）
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_array)

    # 各カラムに目標スケールを適用
    for col_idx, col_name in enumerate(columns):
        if col_name in target_scales:
            # 目標スケール（標準偏差）を適用
            X_scaled[:, col_idx] *= target_scales[col_name]

    # DataFrameで返すかnumpy配列で返すか
    if is_dataframe:
        return pd.DataFrame(X_scaled, columns=columns, index=X.index)
    return X_scaled

target_max = {'bn_price':1,'hdl_price':2,'hdn_price':3,'week_price':10}
X_scaled = custom_scale(X, target_max)

print(X_scaled)

from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test = train_test_split(X_scaled,y,random_state=3)
#X_train,X_test, y_train, y_test = train_test_split(X,y,random_state=0)



       bn_price  hdl_price  hdn_price  week_price
0     -0.080671  -1.655024  -2.595993    3.885479
1     -0.080671  -0.242026  -1.126340   13.382903
2     -0.080671  -1.301774  -2.228580   12.195725
3     -0.080671  -1.184025   7.967138  -26.981150
4     -0.080671  -1.184025   7.967138  -17.483726
...         ...        ...        ...         ...
51630 -0.080671  -0.359776  -1.493754  -12.735014
51631 -0.080671  -0.595275  -0.024100  -12.735014
51632 -0.080671  -1.066275   5.119685   -7.986302
51633 -0.080671  -0.300901  -1.493754    7.565731
51634 -0.080671  -0.300901  -2.412287    3.885479

[51635 rows x 4 columns]


In [ ]:
# prompt: X_train,X_test, y_train, y_test を勾配ブースティングでテストしSMAPEスコアを確認

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import make_scorer, mean_absolute_percentage_error
import numpy as np
# Define SMAPE scorer
def smape(y_true, y_pred):
    return np.mean((2 * np.abs(y_pred - y_true)) / (np.abs(y_true) + np.abs(y_pred))) * 100

smape_scorer = make_scorer(smape, greater_is_better=False)

# Initialize and train the Gradient Boosting Regressor
gbr = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=0)
gbr.fit(X_train, y_train.values.ravel()) # Flatten y_train

# Make predictions
y_pred = gbr.predict(X_test)

# Calculate and print the SMAPE score
smape_score = smape(y_test.values.ravel(), y_pred) # Flatten y_test and y_pred
print(f"SMAPE score: {smape_score:.2f}")


SMAPE score: 64.75


In [ ]:
# prompt: X_train,X_test, y_train, y_test をランダムフォレストでテストしSMAPEスコアを確認

from sklearn.ensemble import RandomForestRegressor

# Initialize and train the Random Forest Regressor
rfr = RandomForestRegressor(n_estimators=100, random_state=0)
rfr.fit(X_train, y_train.values.ravel()) # Flatten y_train

# Make predictions
y_pred_rf = rfr.predict(X_test)

# Calculate and print the SMAPE score for Random Forest
smape_score_rf = smape(y_test.values.ravel(), y_pred_rf) # Flatten y_test and y_pred_rf
print(f"Random Forest SMAPE score: {smape_score_rf:.2f}")

Random Forest SMAPE score: 60.64


In [ ]:
# prompt: X_train,X_test, y_train, y_test を決定木でテストしSMAPEスコアを確認

from sklearn.tree import DecisionTreeRegressor

# Initialize and train the Decision Tree Regressor
dtr = DecisionTreeRegressor(random_state=0)
dtr.fit(X_train, y_train.values.ravel()) # Flatten y_train

# Make predictions
y_pred_dt = dtr.predict(X_test)

# Calculate and print the SMAPE score for Decision Tree
smape_score_dt = smape(y_test.values.ravel(), y_pred_dt) # Flatten y_test and y_pred_dt
print(f"Decision Tree SMAPE score: {smape_score_dt:.2f}")


Decision Tree SMAPE score: 62.82
